#### Run: Viro3. The Viro strikes back

```
Lead     : ababaian
Issue    : 
Version  : v0.3.4
start    : 2020 06 25
complete : 2020 06 25
files    : ~/serratus/notebook/200625_ab/
s3_files : s3://serratus-public/notebook/200625_ab/
output   : s3://serratus-public/out/200625_viro3/
```

### Objectives
- Analyze all "virus" annotated datasets, no restrictions


## SRA Access - Batman returns (Chiroptera)

Search:
```
txid10239[Organism:exp] 
```

Accessed: `2020/06/24`

Results : `121766` saved as `viro3_SraRunInfo.csv`


### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200625_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200625_ab/'

# date and version
date
git rev-parse HEAD # commit version

Wed Jun 24 22:47:33 PDT 2020
7c0d52fd7c62ed82054ec365fc32c99da382c435


## SRA Accession Initialization



### Remove previously completed accessions

In [2]:
cd $WORK
# Data release 200623
aws s3 cp s3://lovelywater/index.tsv ./

download: s3://lovelywater/index.tsv to ./index.tsv   


In [3]:
cat index.tsv \
  | sed 's/^...............................//g' - \
  | cut -f1 -d'.' - > sra.complete
  
wc -l sra.complete

3837755 sra.complete


In [4]:
head -n1 viro3_SraRunInfo.csv > sra.header
grep -vif sra.complete viro3_SraRunInfo.csv > v2_viro3_SraRunInfo.csv

wc -l *.csv

    91226 v2_viro3_SraRunInfo.csv
   122038 viro3_SraRunInfo.csv
   213264 total


### Terraform Initialize

In [4]:
# For rapid batching; copy out serratus folder
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..80ca4d6 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.4xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type  

In [5]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [6]:
wc -l $WORK/v2_viro3_SraRunInfo.csv

91226 /home/artem/serratus/notebook/200625_ab/v2_viro3_SraRunInfo.csv


In [8]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/v2_viro3_SraRunInfo.csv

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200625_ab/v2_viro3_SraRunInfo.csv
  date: Wed Jun 24 23:28:51 PDT 2020
  wc  : 91226 /home/artem/serratus/notebook/200625_ab/v2_viro3_SraRunInfo.csv
  md5 : bbef109b8bb556eaea7bf0e4ac65048b  /home/artem/serratus/notebook/200625_ab/v2_viro3_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
7f597d358da376d8c4695c5980b0e89d  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
dcf03c4de97d7c9d69b42fcb8e8beb2f  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
0849049f56e60d712d945229b8bc67b6  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
e93b4b32b1d23aeda7c12c3616825986  tmp.chunk03_sraRunInfo.csv
{"inserte

## Run Serratus

In [14]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
#
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 50,
  "ALIGN_SCALING_CONSTANT": 0.25,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 20,
  "CLEAR_INTERVAL": 600000,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 15,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "cov3ma",
  "MERGE_ARGS": "",
  "MERGE_MAX_INCREASE": 20,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 10,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":50,"ALIGN_SCALING_CONSTANT":0.25,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":20,"CLEAR_INTERVAL":600000,"DL_ARGS":"","DL_MAX_INCREASE"

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids